In [1]:
!pip3 install -q ktrain

     |████████████████████████████████| 25.3 MB 1.5 MB/s 
     |████████████████████████████████| 22.3 MB 1.5 MB/s 
     |████████████████████████████████| 981 kB 53.3 MB/s 
     |████████████████████████████████| 263 kB 71.1 MB/s 
     |████████████████████████████████| 2.8 MB 60.7 MB/s 
     |████████████████████████████████| 1.2 MB 61.9 MB/s 
     |████████████████████████████████| 468 kB 68.8 MB/s 
     |████████████████████████████████| 880 kB 59.2 MB/s 
     |████████████████████████████████| 3.3 MB 57.6 MB/s 
     |████████████████████████████████| 596 kB 71.5 MB/s 
     |████████████████████████████████| 77 kB 8.8 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.4 requires scikit-learn>=1.0.0, but you have scikit-learn 0.24.2 which is incompatible.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import pandas as pd

In [4]:
data=pd.read_excel("/content/drive/MyDrive/Bert/Release_Notes_v0_RawData.xlsx")

In [5]:
data.head()

,ID,Project_Name,Release_Number,Date,Label,Type,Release_Note
0,1.0,Mozilla Firefox,86.0,2021-02-23,new,new,Firefox now supports simultaneously watching m...
1,2.0,Mozilla Firefox,86.0,2021-02-23,fixed,fixed,Reader mode now works with local HTML pages. U...
2,3.0,Mozilla Firefox,86.0,2021-02-23,changed,changed,"On Linux and Android, the protection to mitiga..."
3,4.0,Mozilla Firefox,86.0,2021-02-23,other,developer,Inactive CSS tool is now showing a warning whe...
4,5.0,Mozilla Firefox,85.0.2,2021-02-09,fixed,fixed,Fixed a deadlock during startup (bug 1679933)


In [6]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.2, random_state=42)

In [7]:
train=train[["Release_Note","Label"]]
test=test[["Release_Note","Label"]]

In [8]:
train["Release_Note"]=train["Release_Note"].apply(lambda r: str(r))
train['Label']=train['Label'].astype(str)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [9]:
train["Release_Note"].head()

264    Column choice and sort order in "Search Messag...
615    Fixed a bug where log files wouldn't be create...
329    Due to changes in the Mozilla platform profile...
342                               Various security fixes
394    Default to using SSL for XMPP and IRC. This mi...
Name: Release_Note, dtype: object

In [10]:
test["Release_Note"].head()

696    Added video/audio filtering; you can now acces...
667    Actually fixes the potential game capture cras...
63     Various stability, functionality, and security...
533    Added the ability to pause while recording to ...
66                                 Developer Information
Name: Release_Note, dtype: object

In [11]:
train["Label"].head()

264         fixed
615       changed
329    unresolved
342         fixed
394       changed
Name: Label, dtype: object

In [12]:
test["Label"].head()

696    changed
667      fixed
63       fixed
533        new
66       other
Name: Label, dtype: object

In [13]:
possible_labels = data.Label.unique().tolist()
possible_labels

['new', 'fixed', 'changed', 'other', 'unresolved']

In [14]:
import ktrain
from ktrain import text

In [15]:
model = ktrain.load_predictor('/content/drive/MyDrive/Bert/bert_model2')

In [38]:
örnek = test.iloc[21]['Release_Note']
print(örnek)

Browser docks now fully shutdown when hidden (prevents playback of any video/audio that may have been playing by browser),Added an option to browser sources, "Reroute audio through OBS", which is enabled by default. This allows the user to turn off the new browser audio handling if they need to for whatever reason (usually only useful when you need to monitor with a different device than what OBS is set to monitor with)


In [39]:
model.predict(örnek)

'changed'

In [40]:
tahmin_deger = model.predict(test['Release_Note'].to_list())

In [41]:
gercek_deger = test["Label"].to_list()

In [42]:
liste = []
for i in range(160):
  if gercek_deger[i] != tahmin_deger[i]:
      print(gercek_deger[i] + " --> " + tahmin_deger[i]+ " --> " + str(i))
      liste.append(i)

fixed --> changed --> 1
new --> changed --> 3
new --> changed --> 17
new --> changed --> 19
fixed --> changed --> 21
changed --> new --> 26
fixed --> new --> 38
fixed --> changed --> 45
fixed --> changed --> 49
new --> changed --> 55
fixed --> changed --> 67
other --> new --> 81
fixed --> changed --> 84
new --> changed --> 87
new --> changed --> 89
fixed --> changed --> 90
new --> changed --> 93
fixed --> new --> 99
new --> changed --> 106
fixed --> changed --> 120
other --> changed --> 123
changed --> new --> 131
other --> changed --> 138
changed --> fixed --> 148
fixed --> changed --> 150


In [43]:
print(liste)

[1, 3, 17, 19, 21, 26, 38, 45, 49, 55, 67, 81, 84, 87, 89, 90, 93, 99, 106, 120, 123, 131, 138, 148, 150]


In [44]:
print(len(liste))

25


In [45]:
model.predict_proba(örnek)

array([0.8973519 , 0.09594491, 0.00224078, 0.00187932, 0.00258321],
      dtype=float32)

In [46]:
model.get_classes()

['changed', 'fixed', 'new', 'other', 'unresolved']

In [47]:
target_names = model.get_classes()

In [48]:
!pip3 install -q git+https://github.com/amaiya/eli5@tfkeras_0_10_1


In [49]:
import eli5
from eli5.lime import TextExplainer

In [50]:
te = TextExplainer(random_state=42)
te.fit(örnek, model.predict_proba)
te.show_prediction(target_names=target_names)

In [51]:
te.show_weights(target_names=target_names, top=10)

In [52]:
te.show_prediction(targets=['changed'],target_names=target_names)

In [53]:
te.show_weights(targets=['changed'],target_names=target_names, top=(5))

In [57]:
örnek="browser docks fully shutdown when hidden (prevents playback of any that may have been playing by browser), an  to browser sources, 'reroute  through obs', which is. this  the user to turn off the new browser handling if they need to for whatever reason (usually only useful when you need to monitor with a different device than what obs is set to monitor with)"

In [58]:
model.predict(örnek)

'fixed'

Yukarda changed sınıfı seçiminde etkin olan(enabled,allows,added,now)ilk dört sözcüğü metinden çıkartıp tekrar modelimize tahmin ettirdiğimizde fixed sınıfını doğru tahmininin yapıldığını gözlemliyoruz.

In [115]:
örnek2 = test.iloc[90]['Release_Note']
print(örnek2)

Added a Gradient feature to the GDI+ text source (not fully localized yet however),Fixed an NVENC crash that could happen occasionally,Fixed a few minor issues with AMD encoder presets,Once again, fixed a bug where the window's position would reset on startup (hopefully)


In [116]:
model.predict(örnek2)

'changed'

In [117]:
te = TextExplainer(random_state=42)
te.fit(örnek2, model.predict_proba)
te.show_prediction(target_names=target_names)

In [118]:
te.show_weights(target_names=target_names, top=10)

In [119]:
te.show_prediction(targets=['changed'],target_names=target_names)

In [120]:
te.show_weights(targets=['changed'],target_names=target_names, top=(5))

In [129]:
örnek2=""" to the GDI+ text(not fully localized yet however),
Fixed an NVENC crash that could happen occasionally,Fixed a few minor issues with 
presets,Once again,a bug  position would reset on startup (hopefully)"""

In [130]:
model.predict(örnek2)

'fixed'

Yukardaki ilk örnek ile benzer uygulamaları örnek2 üzerinde uyguladığımızda benzer şekilde model tahminin doğru tahmin ettiğini görüyoruz.